In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
full_data = pd.read_csv("../../data/processed/historical/train/lcl_data.csv")
df_100K = full_data.sample(100000, random_state=0)
df_100K.to_csv("../../data/processed/historical/train/lcl_data_100K.csv", index=False)

# Load Data

In [3]:
import torch
import numpy as np
import random
RANDOM_STATE = 0
torch.manual_seed(RANDOM_STATE)
torch.use_deterministic_algorithms(True)
g = torch.Generator()
g.manual_seed(RANDOM_STATE)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [4]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data_100K.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=100000, n_samples=100000)
dm.setup()

In [5]:
import torch
from opensynth.models.faraday import FaradayVAE
vae_model = torch.load("vae_model.pt")
vae_model.eval()

FaradayVAE(
  (encoder): Encoder(
    (encoder_layers): Sequential(
      (0): Linear(in_features=50, out_features=512, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): GELU(approximate='none')
      (4): Linear(in_features=256, out_features=128, bias=True)
      (5): GELU(approximate='none')
      (6): Linear(in_features=128, out_features=64, bias=True)
      (7): GELU(approximate='none')
      (8): Linear(in_features=64, out_features=32, bias=True)
      (9): GELU(approximate='none')
      (10): Linear(in_features=32, out_features=16, bias=True)
    )
  )
  (decoder): Decoder(
    (latent): Linear(in_features=18, out_features=16, bias=True)
    (latent_activations): GELU(approximate='none')
    (decoder_layers): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): GELU(approximate='no

In [6]:
from opensynth.models.faraday.gaussian_mixture.prepare_gmm_input import encode_data_for_gmm

next_batch = next(iter(dm.train_dataloader()))
input_tensor = encode_data_for_gmm(data=next_batch, vae_module=vae_model)
input_data = input_tensor.detach().numpy()
n_samples = len(input_tensor)

In [7]:
N_COMPONENTS = 700
REG_COVAR = 1e-4
EPOCHS = 25
IDX = 0
CONVERGENCE_TOL = 1e-2


In [8]:
input_tensor.shape, input_tensor[0][0]

(torch.Size([100000, 18]), tensor(0.4973, grad_fn=<SelectBackward0>))

# Init GMM

In [9]:
from opensynth.models.faraday.new_gmm import gmm_utils

labels_, means_, responsibilities_ = gmm_utils.initialise_centroids(
        X=input_data, n_components=N_COMPONENTS
    )
print(labels_.dtype, responsibilities_.dtype, means_.dtype)

torch.float32 torch.float32 torch.float32


In [10]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params

gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
print(gmm_init_params["precision_cholesky"][IDX][0][0])
print(gmm_init_params["weights"].sum())

tensor(8.1619)
tensor(1.)


# Torch Lightning Batch Learning

In [11]:
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule
class CustomDataset(Dataset):
    def __init__(self, data_tensor: torch.Tensor):
        self.data = data_tensor
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    
class CustomDataModule(LightningDataModule):
    def __init__(self, data_tensor: torch.Tensor, batch_size: int):
        super().__init__()
        self.data_tensor = data_tensor
        self.batch_size = batch_size
    def setup(self, stage=""):
        self.custom_ds = CustomDataset(self.data_tensor)
    def train_dataloader(self):
        return DataLoader(self.custom_ds, batch_size=self.batch_size, shuffle=False, generator=g, worker_init_fn=seed_worker)
    
custom_dm = CustomDataModule(data_tensor=input_tensor, batch_size=25000)
custom_dm.setup(stage="")

In [12]:
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureLightningModule, GaussianMixtureModel
gmm_module = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
gmm_module.initialise(gmm_init_params)
print(f"Initial prec chol: {gmm_module.precision_cholesky[IDX][0][0]}. Initial mean: {gmm_module.means[IDX][0]}")

gmm_lightning_module = GaussianMixtureLightningModule(
    gmm_module = gmm_module,
    vae_module = vae_model,
    num_components = gmm_module.num_components,
    num_features = gmm_module.num_features,
    reg_covar = gmm_module.reg_covar,
    convergence_tolerance = CONVERGENCE_TOL,
    compute_on_batch=False
)
trainer = pl.Trainer(max_epochs=EPOCHS, accelerator="cpu", deterministic=True )
trainer.fit(gmm_lightning_module, custom_dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/opt

Initial prec chol: 8.161928176879883. Initial mean: -0.4217256009578705
Epoch 0: 100%|██████████| 4/4 [00:07<00:00,  0.54it/s, v_num=89]Local weights at rank: 0 - means: 0.0057, -0.4038
Reduced weights, means, covar: 0.0057, -0.4038, 0.0061
log prob:  tensor(3.0437)
Epoch 1: 100%|██████████| 4/4 [00:06<00:00,  0.66it/s, v_num=89]Local weights at rank: 0 - means: 0.0046, -0.3867
Reduced weights, means, covar: 0.0046, -0.3867, 0.0049
log prob:  tensor(2.4145)
Epoch 2: 100%|██████████| 4/4 [00:06<00:00,  0.63it/s, v_num=89]Local weights at rank: 0 - means: 0.0063, -0.3767
Reduced weights, means, covar: 0.0063, -0.3767, 0.0039
log prob:  tensor(2.2645)
Epoch 3: 100%|██████████| 4/4 [00:06<00:00,  0.62it/s, v_num=89]Local weights at rank: 0 - means: 0.0077, -0.3818
Reduced weights, means, covar: 0.0077, -0.3818, 0.0036
log prob:  tensor(2.2010)
Epoch 4: 100%|██████████| 4/4 [00:05<00:00,  0.77it/s, v_num=89]Local weights at rank: 0 - means: 0.0095, -0.3878
Reduced weights, means, covar: 0.0

## SK-learn batch learning

In [14]:
from sklearn.mixture import GaussianMixture
init_weights = gmm_init_params["weights"]
init_means = gmm_init_params["means"]

skgmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='full', tol=CONVERGENCE_TOL, max_iter=EPOCHS, random_state=0, means_init = init_means, weights_init=init_weights, warm_start=True, verbose=1)

dl = custom_dm.train_dataloader()
next_batch = next(iter(dl))
for batch_num, batch_data in enumerate(dl):
    print("Batch number: ", batch_num)
    input_data = batch_data.detach().numpy()
    n_samples = len(input_tensor)
    skgmm.fit(input_data)
    print("means: ", skgmm.means_)


Batch number:  0
Initialization 0
  Iteration 10
  Iteration 20
Initialization converged.
means:  [[-0.29321853 -1.81423416  0.57233811 ... -1.75693345  8.25643716
   3.72416524]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.59857156 -1.27148646  0.53578165 ... -0.90004716  4.94579757
   3.17333562]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
Batch number:  1
Initialization 0
  Iteration 10
Initialization converged.
means:  [[-0.28296558 -1.8220859   0.5540677  ... -1.68444831  7.63053112
   3.19461812]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.57580652 -1.14720437  0.4195868  ... -0.91864648 

In [15]:
gmm_lightning_module.gmm_module.means

tensor([[-0.3989, -2.2289,  0.9466,  ..., -2.3890,  6.8711,  3.6394],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.4233, -1.6980,  0.4012,  ..., -1.4875,  3.9979,  3.4593],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [16]:
skgmm.means_

array([[-0.29255825, -1.66217373,  0.48821718, ..., -1.50956643,
         8.05414594,  3.15504887],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5122906 , -1.18503505,  0.43653833, ..., -1.00035921,
         6.9263532 ,  2.79924122],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [17]:
sklearn_sum_components = skgmm.means_.sum(axis=1)
ligthning_sum_components = gmm_lightning_module.gmm_module.means.sum(axis=1)
len(sklearn_sum_components[sklearn_sum_components==0]), len(ligthning_sum_components[ligthning_sum_components==0])

(609, 542)

# Compare

In [18]:
IDX = 0

In [19]:
df_compare_means = pd.DataFrame()
df_compare_means["skgmm"] = skgmm.means_[IDX]
df_compare_means["lightning"] = gmm_lightning_module.gmm_module.means[IDX]
df_compare_means

,skgmm,lightning
0,-0.292558,-0.398866
1,-1.662174,-2.228910
2,0.488217,0.946641
3,-0.381743,-0.679431
4,-0.288337,0.170474
5,-0.051535,-0.050671
6,0.612039,1.003096
7,1.450273,1.151517
8,0.028997,0.092867
9,-1.285075,-2.511037


In [20]:
df_compare_covar = pd.DataFrame()
df_compare_covar["skgmm"] = skgmm.covariances_[IDX][0]
df_compare_covar["lightning"] = gmm_lightning_module.gmm_module.covariances.detach().numpy()[IDX][0]
df_compare_covar

,skgmm,lightning
0,0.010253,0.003480
1,-0.036115,0.003735
2,-0.006396,-0.002560
3,0.007696,0.002829
4,0.001869,0.001791
5,0.004068,0.001117
6,-0.004528,-0.001428
7,0.011117,-0.003705
8,-0.000008,-0.000752
9,-0.014437,0.008082


In [21]:
df_compare_pre_chol = pd.DataFrame()
df_compare_pre_chol["skgmm"] = skgmm.precisions_cholesky_[IDX][0]
df_compare_pre_chol["lightning"] = gmm_lightning_module.gmm_module.precision_cholesky.detach().numpy()[IDX][0]
df_compare_pre_chol

,skgmm,lightning
0,9.876064,16.951376
1,9.078901,-3.396874
2,5.226848,6.786617
3,-4.028690,-5.933033
4,-3.692206,0.638962
5,-11.422013,-8.078204
6,4.621232,0.309509
7,8.530866,1.683192
8,0.456304,0.919062
9,-6.881251,-7.021809


In [22]:
df_compare_weights = pd.DataFrame()
df_compare_weights["skgmm"] = skgmm.weights_[:10]
df_compare_weights["lightning"] = gmm_lightning_module.gmm_module.weights.detach().numpy()[:10]
df_compare_weights

,skgmm,lightning
0,2.026394e-02,1.120373e-02
1,8.881784e-20,4.768371e-12
2,8.881784e-20,4.768371e-12
3,8.881784e-20,4.768371e-12
4,8.881784e-20,4.768371e-12
5,8.881784e-20,5.990161e-03
6,8.881784e-20,4.768371e-12
7,8.881784e-20,4.768371e-12
8,8.881784e-20,4.768371e-12
9,8.881784e-20,4.768371e-12
